In [1]:
import librosa
import numpy as np
import pandas as pd
import sys
import os
import re
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
path=r"D:\PRML\train\train"
german_path=r"D:\PRML\train\train\Eng"
# train_path=
# test_path="

In [4]:



os.listdir(path)
for file in os.listdir(path):
    file_path=os.path.join(path,file)
    if(re.match("de",file)):
        os.rename(file_path,os.path.join(german_path,file))
    

    

In [ ]:
true_values=[]
features_german=[]
features_english=[]
features_spanish=[]
sf=22050
n_components=32
max_iterations=100
hop_len=int(0.01*sf)   ## hop length in milliseconds 
win_len=int(0.02*sf)   ## 20ms window length
for folder in os.listdir(train_path):
    folder_path=os.path.join(folder,file)
    for file in os.listdir(folder_path):
            
            file_path=os.path.join(folder_path,file)
            arr,sr=librosa.load(file_path,sr=sf)
            arr=arr-np.mean(arr)
            max = np.abs(arr).max()  
            arr=arr/(1.01*max)
            MFCC=librosa.feature.mfcc(arr,sr=sf,n_mfcc=14,dct_type=3,hop_length=hop_len, win_length=win_len,window='hann', n_mels=24)
            MFCC=MFCC[1:14,:]
            MFCC = np.reshape(MFCC,(len(MFCC[0]),len(MFCC)))
            DMFCC=librosa.feature.delta(MFCC,order=1)
            # BFCC=spafe.features.bfcc.bfcc(arr, fs=sf, num_ceps=13, win_len=win_len, win_hop=hop_len, win_type='hamming', nfilts=26, dct_type=3)
            # BFCC=np.array(BFCC)
            # print(BFCC.shape)
            print(MFCC.shape)
            Feature=np.concatenate((MFCC,DMFCC),axis=1)
            if(re.match("de",file)):
                features_german=np.concatenate((features_german,Feature),axis=0)
                true_values.append(0)
            elif(re.match("en",file)):
                features_english=np.concatenate((features_english,Feature),axis=0)
                true_values.append(1)
            elif(re.match("es",file)):
                features_spanish=np.concatenate((features_spanish,Feature),axis=0)
                true_values.append(2)

german_model=GaussianMixture(n_components=n_components,covariance_type='diag',max_iter=max_iterations).fit(features_german)
english_model=GaussianMixture(n_components=n_components,covariance_type='diag',max_iter=max_iterations).fit(features_english)
spanish_model=GaussianMixture(n_components=n_components,covariance_type='diag',max_iter=max_iterations).fit(features_spanish)

print("Training Done")
        


In [ ]:
Scores=[]
for folder in os.listdir(test_path):
    for file in os.listdir(folder):
        temp_score=[]
        file_path=os.path.join(folder,file)
        arr,sr=librosa.load(file_path,sr=sf)
        arr=arr-np.mean(arr)
        max = np.abs(arr).max()  
        arr=arr/(1.01*max)
        MFCC=librosa.feature.mfcc(arr,sr=sf,n_mfcc=14,dct_type=3,hop_length=hop_len, win_length=win_len,window='hann', n_mels=24)
        MFCC=MFCC[1:14,:]
        MFCC = np.reshape(MFCC,(len(MFCC[0]),len(MFCC)))
        DMFCC=librosa.feature.delta(MFCC,order=1)
        # BFCC=spafe.features.bfcc.bfcc(arr, fs=sf, num_ceps=13, win_len=win_len, win_hop=hop_len, win_type='hamming', nfilts=26, dct_type=3)
        # BFCC=np.array(BFCC)
        # print(BFCC.shape)
        print(MFCC.shape)
        Feature=np.concatenate((MFCC,DMFCC),axis=1)
        temp_score.append(german_model.score(Feature))
        temp_score.append(english_model.score(Feature))
        temp_score.append(spanish_model.score(Feature))
        Scores.append(temp_score)
    

